In [401]:
from CRTdb3 import *
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from PredictValues import *
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels import regression
from sklearn import metrics
from sklearn import grid_search
from sklearn.grid_search import GridSearchCV
import datetime

In [402]:
ticker = ['S_FRONT','CA-AMNT_TOT','UAN_TOT','UAN_TOT','S_FRONT2','C_FRONT','W_FRONT','UREA_GRAN','DAP_TOT'] 
field_name = ['CLOSE','FLAT_CIF_DEU','FLAT_FOB_NOLA','FLAT_FOB_NOLA_BID','CLOSE','CLOSE','CLOSE','FLAT_FOB_NOLA','FLAT_FOB_NOLA']
source = ['EXCH','CRU','CRU','GREEN-MARKETS','EXCH','EXCH','EXCH','CRU','CRU']

start_date = '01/05/2014'
end_date = '02/09/2017'
cal_type = 'WD'

crtConn = crtDbOpen()
df = pd.DataFrame()
whole_dataframes = {}
    
for i in range(0,len(ticker)):
            
    period_series_dict = {}                    
    statement ="Select * FROM dbo.CRTGetPeriodSeries('" + start_date + "' , '" + end_date + "' , '" + ticker[i] + "' , '" + field_name[i] + "' , '" + source[i] + "','" + cal_type + "')"
    get_period_series_list = crt_gen_sel (crtConn, statement, None,case="same")
        
    for row in get_period_series_list:
        dt = str(row[0])
        tmpDate = dt.split(' ')
        dt = tmpDate[0]           
        dt = datetime.datetime.strptime(dt,'%Y-%m-%d')
        t_year = dt.year            
        dbDate = str(dt).split(' ')
        dbDate = dbDate[0]
        dbDate = str(dbDate[:len(dbDate)-2] + str(t_year))           
        period_series_dict[dt] = row[1]
            
    statement = " TICKER like '" + ticker[i] + "' AND FIELDNAME = '" + field_name[i] + "' AND DATASOURCE = '" + source[i] + "'"
    TableName = 'CRTTICKERNAME'
    IdColName = 'tickerid'            
    get_id = crt_select_multiple_value(crtConn,TableName, IdColName, statement)            
    statement = " tickerid = " + get_id.get(1)
    statement = statement.replace(',','')
    statement = statement.strip()            
    TableName = 'CRTTICKERVALUE'
    IdColName = 'tickerValueDate,tickerValue'
    PP = crt_select_multiple_value(crtConn,TableName, IdColName, statement)        
    date_list = []
    value_list = []
            
    for key, value in iter(PP.items()):
        temp_value = PP.get(key)
        temp_value = temp_value.split(',')        
        dbValue = float(temp_value[1])
        dbDate = str(temp_value[0])
        dbDate = dbDate.split(' ')
        dbDate = dbDate[0]        
        tmpDate = datetime.datetime.strptime(dbDate,'%Y-%m-%d')               
        t_year = tmpDate.year                
        dbDate = str(dbDate[:len(dbDate)-2] + str(t_year))
                
        if period_series_dict.get(tmpDate):
            period_series_dict[tmpDate] = dbValue
        
    date_list = []
    value_list = [] 
    for key in sorted(period_series_dict.keys()):                
        dbDate = key
        dbValue = period_series_dict.get(key)                
        date_list.append(dbDate)
        value_list.append(dbValue)
        whole_dataframes[i] =  pd.DataFrame()
        
    s_PP = pd.Series(date_list)
    s_PP2 = pd.Series(value_list)
           
    whole_dataframes[i]['Date'] = s_PP
    whole_dataframes[i]['Date'] = pd.to_datetime(whole_dataframes[i]['Date']) #run
    whole_dataframes[i][ticker[i]+'_'+field_name[i]+'_'+source[i]] = s_PP2
            
whole_dataframes[0].index = whole_dataframes[0]['Date'].values  
whole_dataframes[0] = whole_dataframes[0].drop(['Date'],axis=1)
          
for i in range(1,len(ticker)):
    whole_dataframes[i].index = whole_dataframes[i]['Date'].values
    whole_dataframes[i] = whole_dataframes[i].drop(['Date'],axis=1)            
    whole_dataframes[0] = whole_dataframes[0].join(whole_dataframes[i])
df = whole_dataframes[0]

#df = df.dropna()
df = df.fillna(df.mean())
date = df.index
dateList = []       
for tmp in date:
    tmpDate = str(tmp)  
    t_year = tmp.year
    t_month = tmp.month
    t_day = tmp.day          
    strDate = str(t_month) + '/' + str(t_day) + '/' + str(t_year)    
    dateList.append(strDate)
       
df['Date'] = dateList
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.tail()

,Date,S_FRONT_CLOSE_EXCH,CA-AMNT_TOT_FLAT_CIF_DEU_CRU,UAN_TOT_FLAT_FOB_NOLA_CRU,UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS,S_FRONT2_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,W_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU,DAP_TOT_FLAT_FOB_NOLA_CRU
2017-02-03,2/3/2017,1027.00,224.606918,6.675314,5.460000,1037.25,365.25,430.25,281.622642,390.468553
2017-02-06,2/6/2017,1036.00,224.606918,6.675314,6.782841,1046.25,363.75,422.50,281.622642,390.468553
2017-02-07,2/7/2017,1042.75,224.606918,6.675314,6.782841,1053.25,368.50,430.75,281.622642,390.468553
2017-02-08,2/8/2017,1058.75,224.606918,6.675314,6.782841,1069.25,370.75,432.50,281.622642,390.468553
2017-02-09,2/9/2017,1050.50,225.000000,5.890000,6.782841,1061.50,369.50,443.50,244.000000,332.500000


In [403]:
initial_columns = list(df.columns)
print(initial_columns)
#so I have one list initial_columns if anything in that list is present in UAN_list I print location indexlist
UAN_list = ['UAN_TOT_FLAT_FOB_NOLA_CRU','UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS','UAN_TOT_FLAT_FOB_NOLA_MID_GREEN-MARKETS',
           'UAN_TOT_FLAT_FOB_NOLA_OFFER_GREEN-MARKETS','UAN_TOT_FLAT_FOB_USA-MW_CRU','UAN_TOT_FLAT_FOB_USA-MW-E_CRU','UAN_TOT_FLAT_FOB_USA-MW-W_CRU']
indexlist = []
for i in range(0,len(UAN_list)):
    if UAN_list[i] in initial_columns:
        indexlist.append(initial_columns.index(UAN_list[i]))
indexlist      #this part works how I want, it printed [3] because 'UAN_TOT_FLAT_FOB_NOLA_CRU' in UAN_list at 3

['Date', 'S_FRONT_CLOSE_EXCH', 'CA-AMNT_TOT_FLAT_CIF_DEU_CRU', 'UAN_TOT_FLAT_FOB_NOLA_CRU', 'UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS', 'S_FRONT2_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'W_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU', 'DAP_TOT_FLAT_FOB_NOLA_CRU']


[3, 4]

In [404]:
#change UAN to be in format of forecasts
nameList = []
for i in range(0,len(UAN_list)):
    if UAN_list[i] in initial_columns:
        nameList.append(UAN_list[i])
for i in range(0,len(nameList)):
    df[nameList[i]] = df[nameList[i]]*32

print(df)

                  Date  S_FRONT_CLOSE_EXCH  CA-AMNT_TOT_FLAT_CIF_DEU_CRU  \
2014-01-06    1/6/2014         1296.750000                    224.606918   
2014-01-07    1/7/2014         1299.500000                    224.606918   
2014-01-08    1/8/2014         1300.750000                    224.606918   
2014-01-09    1/9/2014         1296.250000                    235.000000   
2014-01-10   1/10/2014         1303.750000                    224.606918   
2014-01-13   1/13/2014         1326.750000                    224.606918   
2014-01-14   1/14/2014         1339.000000                    224.606918   
2014-01-15   1/15/2014         1318.000000                    224.606918   
2014-01-16   1/16/2014         1315.000000                    237.500000   
2014-01-17   1/17/2014         1316.500000                    224.606918   
2014-01-20   1/20/2014         1057.654609                    224.606918   
2014-01-21   1/21/2014         1280.500000                    224.606918   
2014-01-22  

In [405]:
#getting last date to be used for forecast date
last = df.last_valid_index()
print(last)
yearlast = last.year
monthlast = last.month
daylast = last.day
print("year:",yearlast)
print("month:",monthlast)
print("day:",daylast)
if len(str(monthlast))==1:
    monthlast2 = "0" + str(monthlast)
else:
    monthlast = monthlast
if len(str(daylast))==1:
    daylast2 = "0" + str(daylast)
else:
    daylast = daylast   
stringdate = str(yearlast)+str(monthlast2)+str(daylast2)
print("stringdate:",stringdate)

2017-02-09 00:00:00
year: 2017
month: 2
day: 9
stringdate: 20170209


In [406]:
#case and source inputs
case_input = "Average"
source_input = "Bloomberg"
source_input_fert = "CHS Supply"
if case_input == "Average" and source_input == "Bloomberg":
    case_input2 = "AVG"
elif case_input == "Average" and source_input == "EXCH":
    case_input2 = "CLOSE"
elif case_input == "Best":
    case_input2 = "HIGH"
else:
    case_input2 = "LOW"
    
if source_input == "Bloomberg":
    source_input2 = "BLOOMBERG"
else:
    source_input2 = "EXCH"

if (case_input == "Average" or case_input == "Best" or case_input =="Worst") and (source_input_fert == "Buckley" or source_input_fert == "CHS Supply" or source_input_fert == "CRU"):
    case_input3 = "PRICE"
elif case_input == "Average" and source_input_fert == "FIS":
    case_input3 = "MID"
elif case_input == "Best" and source_input_fert == "FIS":
    case_input3 = "OFFER"
else:
    case_input3 = "BID"
    
if source_input_fert == "Buckley":
    source_input_fert2 = "BUCKLEY"
elif source_input_fert == "CHS Supply":
    source_input_fert2 = "CHS"   
elif source_input_fert == "CRU":
    source_input_fert2 = "CRU"    
else:
    source_input_fert2 = "FIS"
print("case_input2:",case_input2)
print("case_input2:",case_input3)
print("source_input2:",source_input2)
print("source_input_fert2:",source_input_fert2)

case_input2: AVG
case_input2: PRICE
source_input2: BLOOMBERG
source_input_fert2: CHS


In [407]:
print("columns:",cols)
x_list = ['S_FRONT_CLOSE_EXCH','C_FRONT_CLOSE_EXCH','UREA_GRAN_FLAT_FOB_NOLA_CRU']
y_list = ['DAP_TOT_FLAT_FOB_NOLA_CRU']
both_list = x_list+y_list
print("x_list:",x_list)
print("y_list:", y_list)
print("both_list:",both_list)

columns: ['Date', 'S_FRONT_CLOSE_EXCH', 'CA-AMNT_TOT_FLAT_CIF_DEU_CRU', 'UAN_TOT_FLAT_FOB_NOLA_CRU', 'UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS', 'S_FRONT2_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'W_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU', 'DAP_TOT_FLAT_FOB_NOLA_CRU']
x_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU']
y_list: ['DAP_TOT_FLAT_FOB_NOLA_CRU']
both_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU', 'DAP_TOT_FLAT_FOB_NOLA_CRU']


In [408]:
commodity_listx = []
commodity_listy = []
for i in range (0,len(x_list)):
    if "C_FRONT" in x_list[i]:
        commodity_listx.append("C")
    elif "S_FRONT" in x_list[i]:
        commodity_listx.append("S") 
    elif "BO_FRONT" in x_list[i]:
        commodity_listx.append("BO")
    elif "W_FRONT" in x_list[i]:
        commodity_listx.append("W")
    elif "CL_FRONT" in x_list[i]:
        commodity_listx.append("CL")
    elif "NG_FRONT" in x_list[i]:
        commodity_listx.append("NG")        
    elif "CT_FRONT" in x_list[i]:
        commodity_listx.append("CT")
    elif "LA_FRONT" in x_list[i]:
        commodity_listx.append("LA")
    elif "COA_FRONT" in x_list[i]:
        commodity_listx.append("COA")        
    elif "XO_FRONT" in x_list[i]:
        commodity_listx.append("XO")        
    elif "HG_FRONT" in x_list[i]:
        commodity_listx.append("HG")        
    elif "GC_FRONT" in x_list[i]:
        commodity_listx.append("GC")        
    elif "HO_FRONT" in x_list[i]:
        commodity_listx.append("HO")
    elif "CO_FRONT" in x_list[i]:
        commodity_listx.append("CO")
    elif "IOE_FRONT" in x_list[i]:
        commodity_listx.append("IOE")
    elif "LL_FRONT" in x_list[i]:
        commodity_listx.append("LL")
    elif "FN_FRONT" in x_list[i]:
        commodity_listx.append("FN")
    elif "LN_FRONT" in x_list[i]:
        commodity_listx.append("LN")
    elif "PA_FRONT" in x_list[i]:
        commodity_listx.append("PA")
    elif "PL_FRONT" in x_list[i]:
        commodity_listx.append("PL")
    elif "XB_FRONT" in x_list[i]:
        commodity_listx.append("XB")
    elif "SI_FRONT" in x_list[i]:
        commodity_listx.append("SI")
    elif "HRC_FRONT" in x_list[i]:
        commodity_listx.append("HRC")
    elif "SB_FRONT" in x_list[i]:
        commodity_listx.append("SB")        
    elif "LT_FRONT" in x_list[i]:
        commodity_listx.append("LT")        
    elif "UXA_FRONT" in x_list[i]:
        commodity_listx.append("UXA")
    elif "LX_FRONT" in x_list[i]:
        commodity_listx.append("LX")        
    else:
        commodity_listx.append("NaN")
for i in range (0,len(y_list)):
    if "C_FRONT" in y_list[i]:
        commodity_listy.append("C")
    elif "S_FRONT" in y_list[i]:
        commodity_listy.append("S") 
    elif "BO_FRONT" in y_list[i]:
        commodity_listy.append("BO")
    elif "W_FRONT" in y_list[i]:
        commodity_listy.append("W")
    elif "CL_FRONT" in y_list[i]:
        commodity_listy.append("CL")
    elif "NG_FRONT" in y_list[i]:
        commodity_listy.append("NG")        
    elif "CT_FRONT" in y_list[i]:
        commodity_listy.append("CT")
    elif "LA_FRONT" in y_list[i]:
        commodity_listy.append("LA")
    elif "COA_FRONT" in y_list[i]:
        commodity_listy.append("COA")        
    elif "XO_FRONT" in y_list[i]:
        commodity_listy.append("XO")        
    elif "HG_FRONT" in y_list[i]:
        commodity_listy.append("HG")        
    elif "GC_FRONT" in y_list[i]:
        commodity_listy.append("GC")        
    elif "HO_FRONT" in y_list[i]:
        commodity_listy.append("HO")
    elif "CO_FRONT" in y_list[i]:
        commodity_listy.append("CO")
    elif "IOE_FRONT" in y_list[i]:
        commodity_listy.append("IOE")
    elif "LL_FRONT" in y_list[i]:
        commodity_listy.append("LL")
    elif "FN_FRONT" in y_list[i]:
        commodity_listy.append("FN")
    elif "LN_FRONT" in y_list[i]:
        commodity_listy.append("LN")
    elif "PA_FRONT" in y_list[i]:
        commodity_listy.append("PA")
    elif "PL_FRONT" in y_list[i]:
        commodity_listy.append("PL")
    elif "XB_FRONT" in y_list[i]:
        commodity_listy.append("XB")
    elif "SI_FRONT" in y_list[i]:
        commodity_listy.append("SI")
    elif "HRC_FRONT" in y_list[i]:
        commodity_listy.append("HRC")
    elif "SB_FRONT" in y_list[i]:
        commodity_listy.append("SB")        
    elif "LT_FRONT" in y_list[i]:
        commodity_listy.append("LT")        
    elif "UXA_FRONT" in y_list[i]:
        commodity_listy.append("UXA")
    elif "LX_FRONT" in y_list[i]:
        commodity_listy.append("LX")        
    else:
        commodity_listy.append("NaN") 
print('x_list:',x_list)       
print("y_list:",y_list)
print("commodity_listx:",commodity_listx)
print("commodity_listy:",commodity_listy)        

x_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU']
y_list: ['DAP_TOT_FLAT_FOB_NOLA_CRU']
commodity_listx: ['S', 'C', 'NaN']
commodity_listy: ['NaN']


In [409]:
commodity_listx2 = []
commodity_listy2 = []
field_listx2 = []
field_listy2 = []
if source_input_fert2 == "CHS" or source_input_fert2 == "BUCKLEY":
    for i in range (0,len(x_list)):
        if "DAP_TOT" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_NOLA")
        elif "DAP_TOT" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_NOLA")            
        elif "DAP_TOT" in x_list[i] and "FOB_NOLA-EXPORT" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_NOLA")            
        elif "DAP_TOT" in x_list[i] and "FOB_MWEST" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_MWEST")
        elif "DAP_TOT" in x_list[i] and "FOB_USA-MW" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_MWEST")            
        elif "POT_TOT" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("POT_TOT")
            field_listx2.append("FOB_NOLA")
        elif "POT_TOT" in x_list[i] and "FOB_MWEST" in x_list[i]:
            commodity_listx2.append("POT_TOT")
            field_listx2.append("FOB_MWEST")  
        elif "POT_TOT" in x_list[i] and "FOB_USA-MW" in x_list[i]:
            commodity_listx2.append("POT_TOT")
            field_listx2.append("FOB_MWEST")              
        elif "UAN_TOT" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_NOLA")
        elif "UAN_TOT" in x_list[i] and "FOB_NOLA-ST" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_NOLA")            
        elif "UAN_32N" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_NOLA")  
        elif "UAN_32N" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_NOLA")             
        elif "UAN_TOT" in x_list[i] and "FOB_MWEST" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_MWEST")  
        elif "UAN_TOT" in x_list[i] and "FOB_USA-MW" in x_list[i]:
            commodity_listx2.append("UAN_TOT")
            field_listx2.append("FOB_MWEST")              
        elif "UREA_GRAN" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_NOLA")
        elif "UREA_GRAN" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_NOLA")            
        elif "UREA_GRAN" in x_list[i] and "FOB_MWEST" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_MWEST")
        elif "UREA_GRAN" in x_list[i] and "FOB_USA-MW" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_MWEST")            
        else:
            commodity_listx2.append("NaN")
            field_listx2.append("NaN")
elif source_input_fert2 == "CRU":
    for i in range (0,len(x_list)):
        if "DAP_TOT" in x_list[i] and "FOB_TAMPA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_TAMPA")
        elif "DAP_TOT" in x_list[i] and "FOB_USG-TAMPA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_TAMPA")            
        elif "UREA_GRAN" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_NOLA")        
        else:
            commodity_listx2.append("NaN")
            field_listx2.append("NaN")
else:            
    for i in range (0,len(x_list)):
        if "DAP_TOT" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_USA_NOLA")
        elif "DAP_TOT" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("DAP_TOT")
            field_listx2.append("FOB_USA_NOLA")  
        elif "UAN_32N" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_32N")
            field_listx2.append("FOB_USA_NOLA")
        elif "UAN_32N" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_32N")
            field_listx2.append("FOB_USA_NOLA")
        elif "UAN_TOT" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UAN_32N")
            field_listx2.append("FOB_USA_NOLA")
        elif "UREA_GRAN" in x_list[i] and "FOB_NOLA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_USA_NOLA")
        elif "UREA_GRAN" in x_list[i] and "FOB_USA_NOLA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_USA_NOLA")   
        elif "UAN_30N" in x_list[i] and "FOT_INT_ROUEN" in x_list[i]:
            commodity_listx2.append("UAN_30N")
            field_listx2.append("FOT_INT_ROUEN")
        elif "UAN_30N" in x_list[i] and "FCA_URO" in x_list[i]:
            commodity_listx2.append("UAN_30N")
            field_listx2.append("FOT_INT_ROUEN") 
        elif "UREA_GRAN" in x_list[i] and "CFR_BRA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("CFR_INT_BRA")   
        elif "UREA_GRAN" in x_list[i] and "CFR_INT_BRA" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("CFR_INT_BRA")   
        elif "UREA_GRAN" in x_list[i] and "FOB_EGY" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_INT_EGY")   
        elif "UREA_GRAN" in x_list[i] and "FOB_INT_EGY" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_INT_EGY") 
        elif "UREA_GRAN" in x_list[i] and "FOB_MEST" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_INT_MEST")   
        elif "UREA_GRAN" in x_list[i] and "FOB_INT_MEST" in x_list[i]:
            commodity_listx2.append("UREA_GRAN")
            field_listx2.append("FOB_INT_MEST") 
        elif "UREA_PRILL" in x_list[i] and "FOB_CHN" in x_list[i]:
            commodity_listx2.append("UREA_PRILL")
            field_listx2.append("FOB_INT_CHN")   
        elif "UREA_PRILL" in x_list[i] and "FOB_INT_CHN" in x_list[i]:
            commodity_listx2.append("UREA_PRILL")
            field_listx2.append("FOB_INT_CHN")  
        elif "UREA_PRILL" in x_list[i] and "FOB_INT_YUZHNY" in x_list[i]:
            commodity_listx2.append("UREA_PRILL")
            field_listx2.append("FOB_INT_YUZHNY")            
        else:
            commodity_listx2.append("NaN")
            field_listx2.append("NaN")
if source_input_fert2 == "CHS" or source_input_fert2 == "BUCKLEY":
    for i in range (0,len(y_list)):
        if "DAP_TOT" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_NOLA")
        elif "DAP_TOT" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_NOLA")            
        elif "DAP_TOT" in y_list[i] and "FOB_NOLA-EXPORT" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_NOLA")            
        elif "DAP_TOT" in y_list[i] and "FOB_MWEST" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_MWEST")
        elif "DAP_TOT" in y_list[i] and "FOB_USA-MW" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_MWEST")            
        elif "POT_TOT" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("POT_TOT")
            field_listy2.append("FOB_NOLA")
        elif "POT_TOT" in y_list[i] and "FOB_MWEST" in y_list[i]:
            commodity_listy2.append("POT_TOT")
            field_listy2.append("FOB_MWEST")  
        elif "POT_TOT" in y_list[i] and "FOB_USA-MW" in y_list[i]:
            commodity_listy2.append("POT_TOT")
            field_listy2.append("FOB_MWEST")              
        elif "UAN_TOT" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_NOLA")
        elif "UAN_TOT" in y_list[i] and "FOB_NOLA-ST" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_NOLA")            
        elif "UAN_32N" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_NOLA")  
        elif "UAN_32N" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_NOLA")             
        elif "UAN_TOT" in y_list[i] and "FOB_MWEST" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_MWEST")  
        elif "UAN_TOT" in y_list[i] and "FOB_USA-MW" in y_list[i]:
            commodity_listy2.append("UAN_TOT")
            field_listy2.append("FOB_MWEST")              
        elif "UREA_GRAN" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_NOLA")
        elif "UREA_GRAN" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_NOLA")            
        elif "UREA_GRAN" in y_list[i] and "FOB_MWEST" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_MWEST")
        elif "UREA_GRAN" in y_list[i] and "FOB_USA-MW" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_MWEST")            
        else:
            commodity_listy2.append("NaN")
            field_listy2.append("NaN")
elif source_input_fert2 == "CRU":
    for i in range (0,len(y_list)):
        if "DAP_TOT" in y_list[i] and "FOB_TAMPA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_TAMPA")
        elif "DAP_TOT" in y_list[i] and "FOB_USG-TAMPA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_TAMPA")            
        elif "UREA_GRAN" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_NOLA")        
        else:
            commodity_listy2.append("NaN")
            field_listy2.append("NaN")
else:            
    for i in range (0,len(y_list)):
        if "DAP_TOT" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_USA_NOLA")
        elif "DAP_TOT" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("DAP_TOT")
            field_listy2.append("FOB_USA_NOLA")  
        elif "UAN_32N" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_32N")
            field_listy2.append("FOB_USA_NOLA")
        elif "UAN_32N" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_32N")
            field_listy2.append("FOB_USA_NOLA")
        elif "UAN_TOT" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UAN_32N")
            field_listy2.append("FOB_USA_NOLA")
        elif "UREA_GRAN" in y_list[i] and "FOB_NOLA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_USA_NOLA")
        elif "UREA_GRAN" in y_list[i] and "FOB_USA_NOLA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_USA_NOLA")   
        elif "UAN_30N" in y_list[i] and "FOT_INT_ROUEN" in y_list[i]:
            commodity_listy2.append("UAN_30N")
            field_listy2.append("FOT_INT_ROUEN")
        elif "UAN_30N" in y_list[i] and "FCA_URO" in y_list[i]:
            commodity_listy2.append("UAN_30N")
            field_listy2.append("FOT_INT_ROUEN") 
        elif "UREA_GRAN" in y_list[i] and "CFR_BRA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("CFR_INT_BRA")   
        elif "UREA_GRAN" in y_list[i] and "CFR_INT_BRA" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("CFR_INT_BRA")   
        elif "UREA_GRAN" in y_list[i] and "FOB_EGY" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_INT_EGY")   
        elif "UREA_GRAN" in y_list[i] and "FOB_INT_EGY" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_INT_EGY") 
        elif "UREA_GRAN" in y_list[i] and "FOB_MEST" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_INT_MEST")   
        elif "UREA_GRAN" in y_list[i] and "FOB_INT_MEST" in y_list[i]:
            commodity_listy2.append("UREA_GRAN")
            field_listy2.append("FOB_INT_MEST") 
        elif "UREA_PRILL" in y_list[i] and "FOB_CHN" in y_list[i]:
            commodity_listy2.append("UREA_PRILL")
            field_listy2.append("FOB_INT_CHN")   
        elif "UREA_PRILL" in y_list[i] and "FOB_INT_CHN" in y_list[i]:
            commodity_listy2.append("UREA_PRILL")
            field_listy2.append("FOB_INT_CHN")  
        elif "UREA_PRILL" in y_list[i] and "FOB_INT_YUZHNY" in y_list[i]:
            commodity_listy2.append("UREA_PRILL")
            field_listy2.append("FOB_INT_YUZHNY")            
        else:
            commodity_listy2.append("NaN") 
            field_listy2.append("NaN")
print('x_list:',x_list)       
print("y_list:",y_list)
print("commodity_listx2:",commodity_listx2)
print("commodity_listy2:",commodity_listy2)            

x_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU']
y_list: ['DAP_TOT_FLAT_FOB_NOLA_CRU']
commodity_listx2: ['NaN', 'NaN', 'UREA_GRAN']
commodity_listy2: ['DAP_TOT']


In [410]:
##what position x variables belong in total df
commodity_listxpos = []
commodity_listypos = []
for i in range (0,len(x_list)):
    if x_list[i] in cols and commodity_listx[i]!="NaN":
        commodity_listxpos.append(both_list.index(x_list[i]))
    else:
        commodity_listxpos.append("NaN")
for i in range (0,len(y_list)):
    if y_list[i] in cols and commodity_listy[i]!="NaN":
        commodity_listypos.append(both_list.index(y_list[i]))
    else:
        commodity_listypos.append("NaN")  
print("both_list:",both_list)
print("commodity_listxpos:",commodity_listxpos)
print("commodity_listypos:",commodity_listypos)

both_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU', 'DAP_TOT_FLAT_FOB_NOLA_CRU']
commodity_listxpos: [0, 1, 'NaN']
commodity_listypos: ['NaN']


In [411]:
##what position x variables belong in total df
commodity_listxpos2 = []
commodity_listypos2 = []
for i in range (0,len(x_list)):
    if x_list[i] in cols and commodity_listx2[i]!="NaN":
        commodity_listxpos2.append(both_list.index(x_list[i]))
    else:
        commodity_listxpos2.append("NaN")
for i in range (0,len(y_list)):
    if y_list[i] in cols and commodity_listy2[i]!="NaN":
        commodity_listypos2.append(both_list.index(y_list[i]))
    else:
        commodity_listypos2.append("NaN")  
print("both_list:",both_list)
print("commodity_listxpos2:",commodity_listxpos2)
print("commodity_listypos2:",commodity_listypos2)

both_list: ['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH', 'UREA_GRAN_FLAT_FOB_NOLA_CRU', 'DAP_TOT_FLAT_FOB_NOLA_CRU']
commodity_listxpos2: ['NaN', 'NaN', 2]
commodity_listypos2: [3]


In [412]:
def remove_values_from_list(the_list, val):
    while val in the_list:
        the_list.remove(val)
if ("NaN" in commodity_listy) and ("NaN" in commodity_listy2):
    print('choose different y value, otherwise will not work')
elif "NaN" in (commodity_listx+commodity_listx2):
    print('nan in x')
    remove_values_from_list(commodity_listx, "NaN")
    remove_values_from_list(commodity_listxpos, "NaN")
    remove_values_from_list(commodity_listx2, "NaN")
    remove_values_from_list(commodity_listxpos2, "NaN")
else:
    pass

if "NaN" in field_listy2:
    print('choose different y value')
elif "NaN" in field_listx2:
    print('nan in x')
    def remove_values_from_list(the_list, val):
        while val in the_list:
            the_list.remove(val)
    remove_values_from_list(field_listx2, "NaN")
else:
    pass

print('commodity_listx:',commodity_listx)
print('commodity_listy:',commodity_listy)
print('commodity_listxpos:',commodity_listxpos)
print('commodity_listypos:',commodity_listypos)
print('commodity_listx2',commodity_listx2)
print('commodity_listy2',commodity_listy2)
print('commodity_listxpos2:',commodity_listxpos2)
print('commodity_listypos2:',commodity_listypos2)
print('field_listx2:',field_listx2)
print('field_listy2:',field_listy2)

nan in x
nan in x
commodity_listx: ['S', 'C']
commodity_listy: ['NaN']
commodity_listxpos: [0, 1]
commodity_listypos: ['NaN']
commodity_listx2 ['UREA_GRAN']
commodity_listy2 ['DAP_TOT']
commodity_listxpos2: [2]
commodity_listypos2: [3]
field_listx2: ['FOB_NOLA']
field_listy2: ['FOB_NOLA']


In [413]:
x_dict = {}
for i in range(0,len(commodity_listx)):
    x_dict[i] = getForecastTable(commodity_listx[i],stringdate,source_input2, case_input2)

dfnew = pd.DataFrame(x_dict)
dfnew

,0,1
2017-03-31,1021.73,357.90
2017-06-30,1017.90,369.67
2017-09-30,1008.20,372.44
2017-12-31,991.00,380.11
2018-03-31,1004.00,382.83
2018-06-30,1033.50,393.17
2018-12-31,1025.11,391.44
2019-12-31,1079.17,414.17
2020-12-31,1091.50,407.25


In [414]:
new_list = []
for i in commodity_listxpos:
    new_list.append(both_list[i])
new_list

['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH']

In [415]:
dfnew.columns = new_list 
dfnew.head()

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH
2017-03-31,1021.73,357.90
2017-06-30,1017.90,369.67
2017-09-30,1008.20,372.44
2017-12-31,991.00,380.11
2018-03-31,1004.00,382.83


In [416]:
x_dict2 = {}
stringdate = "20160430"
#take into account if user chose date too far back before things were forecasted
if source_input_fert2 == "BUCKLEY":
    if stringdate < "20161107":
        stringdate = "20161107"
    else:
        stringdate = stringdate
elif source_input_fert2 == "CHS":
    if stringdate < "20161222":
        stringdate = "20161222"
    else: 
        stringdate = stringdate
elif source_input_fert2 == "CRU":
    if stringdate < "20130930":
        stringdate = "20130930"
    else:
        stringdate = stringdate
else:
    stringdate = stringdate
for i in range(0,len(commodity_listx2)):        
    x_dict2[i] = getForecastTable(commodity_listx2[i],stringdate,source_input_fert2, case_input3,field_listx2[i])

dfnew2 = pd.DataFrame(x_dict2)
dfnew2

,0
2017-01-31,235
2017-02-28,240
2017-03-31,245
2017-04-30,245
2017-05-31,235
2017-06-30,220
2017-07-31,210
2017-08-31,195
2017-09-30,190
2017-10-31,190


In [417]:
new_list2 = []
for i in commodity_listxpos2:
    new_list2.append(both_list[i])
new_list2

['UREA_GRAN_FLAT_FOB_NOLA_CRU']

In [418]:
dfnew2.columns = new_list2 
dfnew2.head()

,UREA_GRAN_FLAT_FOB_NOLA_CRU
2017-01-31,235
2017-02-28,240
2017-03-31,245
2017-04-30,245
2017-05-31,235


In [419]:
initial_combined = dfnew.join(dfnew2)
initial_combined

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2017-03-31,1021.73,357.90,245
2017-06-30,1017.90,369.67,220
2017-09-30,1008.20,372.44,190
2017-12-31,991.00,380.11,210
2018-03-31,1004.00,382.83,NaN
2018-06-30,1033.50,393.17,NaN
2018-12-31,1025.11,391.44,NaN
2019-12-31,1079.17,414.17,NaN
2020-12-31,1091.50,407.25,NaN


In [420]:
df_cagr = df[x_list]
df_cagr.head()

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2014-01-06,1296.75,427.75,281.622642
2014-01-07,1299.50,426.00,281.622642
2014-01-08,1300.75,417.00,281.622642
2014-01-09,1296.25,412.00,377.500000
2014-01-10,1303.75,432.75,281.622642


In [421]:
cols2 = df_cagr.columns
print(cols2)
print(len(cols2))

Index(['S_FRONT_CLOSE_EXCH', 'C_FRONT_CLOSE_EXCH',
       'UREA_GRAN_FLAT_FOB_NOLA_CRU'],
      dtype='object')
3


In [422]:
frames22 = [df_cagr,initial_combined]
combined22 = pd.concat(frames22)
combined22

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2014-01-06,1296.750000,427.75000,281.622642
2014-01-07,1299.500000,426.00000,281.622642
2014-01-08,1300.750000,417.00000,281.622642
2014-01-09,1296.250000,412.00000,377.500000
2014-01-10,1303.750000,432.75000,281.622642
2014-01-13,1326.750000,434.50000,281.622642
2014-01-14,1339.000000,431.50000,281.622642
2014-01-15,1318.000000,425.75000,281.622642
2014-01-16,1315.000000,428.00000,391.500000
2014-01-17,1316.500000,424.00000,281.622642


In [218]:
#datastart = {}
#dataend = {}
#for i in range(0,len(cols2)):
#    datastart[i] = (combined22[[i]].dropna()).index[0]
#    dataend[i] = (combined22[[i]].dropna()).index[-1]
#print(datastart)
#print(dataend)

{0: Timestamp('2014-01-09 00:00:00'), 1: Timestamp('2014-01-09 00:00:00'), 2: Timestamp('2014-01-09 00:00:00')}
{0: Timestamp('2020-12-31 00:00:00'), 1: Timestamp('2020-12-31 00:00:00'), 2: Timestamp('2017-12-31 00:00:00')}


In [222]:
#dataend.get(2)

Timestamp('2017-12-31 00:00:00')

In [224]:
#days = (df_cagr.index[-1] - df_cagr.index[0]).days
#cols2 = df_cagr.columns
#dictzz = {}
#for i in range(0,len(cols2)):
#    dictzz[i] = str(round(((((df_cagr[cols2[i]][-1]) / df_cagr[cols2[i]][1])) ** (365.0/((dataend.get(i)-datastart.get(i)).days))) - 1,4)*100)
#dictzz    

{0: '-3.26', 1: '-1.93', 2: '-11.99'}

In [423]:
initial_combined

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2017-03-31,1021.73,357.90,245
2017-06-30,1017.90,369.67,220
2017-09-30,1008.20,372.44,190
2017-12-31,991.00,380.11,210
2018-03-31,1004.00,382.83,NaN
2018-06-30,1033.50,393.17,NaN
2018-12-31,1025.11,391.44,NaN
2019-12-31,1079.17,414.17,NaN
2020-12-31,1091.50,407.25,NaN


In [424]:
combined22.tail()

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2018-03-31,1004.00,382.83,NaN
2018-06-30,1033.50,393.17,NaN
2018-12-31,1025.11,391.44,NaN
2019-12-31,1079.17,414.17,NaN
2020-12-31,1091.50,407.25,NaN


In [425]:
# parse and make a numpy array
#data = np.array( [[np.nan if aaa=='NaN' else int(aaa) for aaa in aa.split('\t')] for aa in a.splitlines()] )
data = combined22.values
for col in range(len(cols2)):

    Nyears = np.isnan(data[:,col]).argmax()-1 # row index for the last non-NaN value
    endvalue = data[Nyears,col]
    startvalue = data[0,col]
    cagr = (endvalue / startvalue) ** (1 / Nyears)
    print ("Nyears:",Nyears, "endval:",endvalue,"cagr:", cagr)

    for year in np.argwhere(np.isnan(data[:,col])):
        data[year,col] = data[year-1,col] * cagr

print (data)

Nyears: -1 endval: 1091.5 cagr: 1.18804397618
Nyears: -1 endval: 407.25 cagr: 1.05033763045
Nyears: 812 endval: 210.0 cagr: 0.99963866075
[[ 1296.75         427.75         281.62264151]
 [ 1299.5          426.           281.62264151]
 [ 1300.75         417.           281.62264151]
 ..., 
 [ 1025.11         391.44         209.77243852]
 [ 1079.17         414.17         209.6966395 ]
 [ 1091.5          407.25         209.62086788]]


In [426]:
ddd = pd.DataFrame(data)
ddd.index = combined22.index
ddd.columns = cols2
ddd

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2014-01-06,1296.750000,427.75000,281.622642
2014-01-07,1299.500000,426.00000,281.622642
2014-01-08,1300.750000,417.00000,281.622642
2014-01-09,1296.250000,412.00000,377.500000
2014-01-10,1303.750000,432.75000,281.622642
2014-01-13,1326.750000,434.50000,281.622642
2014-01-14,1339.000000,431.50000,281.622642
2014-01-15,1318.000000,425.75000,281.622642
2014-01-16,1315.000000,428.00000,391.500000
2014-01-17,1316.500000,424.00000,281.622642


In [427]:
ddd = np.round(ddd,2)
ddd

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2014-01-06,1296.75,427.75,281.62
2014-01-07,1299.50,426.00,281.62
2014-01-08,1300.75,417.00,281.62
2014-01-09,1296.25,412.00,377.50
2014-01-10,1303.75,432.75,281.62
2014-01-13,1326.75,434.50,281.62
2014-01-14,1339.00,431.50,281.62
2014-01-15,1318.00,425.75,281.62
2014-01-16,1315.00,428.00,391.50
2014-01-17,1316.50,424.00,281.62


In [428]:
forecastdatestart = initial_combined.first_valid_index()
forecastdateend = initial_combined.last_valid_index()
print(forecastdatestart)
print(forecastdateend)

2017-03-31 00:00:00
2020-12-31 00:00:00


In [429]:
dddd = ddd[forecastdatestart:forecastdateend]
dddd

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2017-03-31,1021.73,357.90,245.00
2017-06-30,1017.90,369.67,220.00
2017-09-30,1008.20,372.44,190.00
2017-12-31,991.00,380.11,210.00
2018-03-31,1004.00,382.83,209.92
2018-06-30,1033.50,393.17,209.85
2018-12-31,1025.11,391.44,209.77
2019-12-31,1079.17,414.17,209.70
2020-12-31,1091.50,407.25,209.62


In [430]:
df_old = df
df_old = df_old.drop(['Date'],axis=1)
df_old.head()

,S_FRONT_CLOSE_EXCH,CA-AMNT_TOT_FLAT_CIF_DEU_CRU,UAN_TOT_FLAT_FOB_NOLA_CRU,UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS,S_FRONT2_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,W_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU,DAP_TOT_FLAT_FOB_NOLA_CRU
2014-01-06,1296.75,224.606918,213.610063,257.920000,1276.75,427.75,605.75,281.622642,390.468553
2014-01-07,1299.50,224.606918,213.610063,217.050909,1276.00,426.00,602.50,281.622642,390.468553
2014-01-08,1300.75,224.606918,213.610063,217.050909,1269.25,417.00,588.75,281.622642,390.468553
2014-01-09,1296.25,235.000000,254.880000,217.050909,1273.75,412.00,584.25,377.500000,382.500000
2014-01-10,1303.75,224.606918,213.610063,256.000000,1278.50,432.75,569.00,281.622642,390.468553


In [431]:
X = df_old[x_list].values #S, C, urea gran
y = df_old[y_list].values #dap
print(X)
print(y)

[[ 1296.75         427.75         281.62264151]
 [ 1299.5          426.           281.62264151]
 [ 1300.75         417.           281.62264151]
 ..., 
 [ 1042.75         368.5          281.62264151]
 [ 1058.75         370.75         281.62264151]
 [ 1050.5          369.5          244.        ]]
[[ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 382.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 432.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 440.        ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 437.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 447.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 485.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 482.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.46855346]
 [ 487.5       ]
 [ 390.46855346]
 [ 390.46855346]
 [ 390.

In [432]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3,random_state=1)
def fit_predict_model(X_train,y_train):  
    regressor = RandomForestRegressor()
    parameters = {'n_estimators':(175,200),'max_depth':(9,10)}
    mse_scorer = metrics.make_scorer(metrics.mean_squared_error, greater_is_better = False)
    reg = grid_search.GridSearchCV(regressor, param_grid=parameters,scoring=mse_scorer, cv = 10)
    reg.fit(X_train, y_train)
    return reg
        
rdf_model_ureaall = fit_predict_model(X_train,y_train)
rdf_model_ureaall.best_estimator_
forest1 = rdf_model_ureaall.best_estimator_ 
forest1.fit(X_train, y_train)   
lm8 = LinearRegression()
lm8.fit(X_train,y_train)
    
model1 = forest1.predict(X)
df_old['RDF Model'] = model1
model2 = lm8.predict(X)
df_old['MLR Model'] = model2
df_old

C:\Users\msteinme\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1665: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\msteinme\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1665: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\msteinme\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:1665: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\msteinme\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\c

,S_FRONT_CLOSE_EXCH,CA-AMNT_TOT_FLAT_CIF_DEU_CRU,UAN_TOT_FLAT_FOB_NOLA_CRU,UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS,S_FRONT2_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,W_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU,DAP_TOT_FLAT_FOB_NOLA_CRU,RDF Model,MLR Model
2014-01-06,1296.750000,224.606918,213.610063,257.920000,1276.7500,427.75000,605.750000,281.622642,390.468553,390.468830,389.663053
2014-01-07,1299.500000,224.606918,213.610063,217.050909,1276.0000,426.00000,602.500000,281.622642,390.468553,390.468830,389.491347
2014-01-08,1300.750000,224.606918,213.610063,217.050909,1269.2500,417.00000,588.750000,281.622642,390.468553,390.468830,388.829230
2014-01-09,1296.250000,235.000000,254.880000,217.050909,1273.7500,412.00000,584.250000,377.500000,382.500000,440.965706,467.138152
2014-01-10,1303.750000,224.606918,213.610063,256.000000,1278.5000,432.75000,569.000000,281.622642,390.468553,390.468830,389.899038
2014-01-13,1326.750000,224.606918,213.610063,260.160000,1294.2500,434.50000,573.500000,281.622642,390.468553,390.468830,389.629471
2014-01-14,1339.000000,224.606918,213.610063,217.050909,1307.0000,431.50000,579.250000,281.622642,390.468553,390.468830,389.205979
2014-01-15,1318.000000,224.606918,213.610063,217.050909,1297.5000,425.75000,567.750000,281.622642,390.468553,390.468830,389.156519
2014-01-16,1315.000000,237.500000,266.880000,217.050909,1296.2500,428.00000,572.750000,391.500000,432.500000,463.336304,479.431226
2014-01-17,1316.500000,224.606918,213.610063,256.000000,1297.2500,424.00000,563.500000,281.622642,390.468553,390.468830,389.057644


In [433]:
dfnew2 = dddd
dfnew2

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
2017-03-31,1021.73,357.90,245.00
2017-06-30,1017.90,369.67,220.00
2017-09-30,1008.20,372.44,190.00
2017-12-31,991.00,380.11,210.00
2018-03-31,1004.00,382.83,209.92
2018-06-30,1033.50,393.17,209.85
2018-12-31,1025.11,391.44,209.77
2019-12-31,1079.17,414.17,209.70
2020-12-31,1091.50,407.25,209.62


In [434]:
dfnew2['Date'] = dfnew2.index
dfnew2 = dfnew2.reset_index(drop=True)
#df3 = dfnew2[['Date']]
#df4 = dfnew2[x_list]
#newpred = df3.join(df4)
#newpred
dfnew2

C:\Users\msteinme\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU,Date
0,1021.73,357.90,245.00,2017-03-31
1,1017.90,369.67,220.00,2017-06-30
2,1008.20,372.44,190.00,2017-09-30
3,991.00,380.11,210.00,2017-12-31
4,1004.00,382.83,209.92,2018-03-31
5,1033.50,393.17,209.85,2018-06-30
6,1025.11,391.44,209.77,2018-12-31
7,1079.17,414.17,209.70,2019-12-31
8,1091.50,407.25,209.62,2020-12-31


In [435]:
df3 = dfnew2[['Date']]
df4 = dfnew2[x_list]
newpred = df3.join(df4)
newpred

,Date,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
0,2017-03-31,1021.73,357.90,245.00
1,2017-06-30,1017.90,369.67,220.00
2,2017-09-30,1008.20,372.44,190.00
3,2017-12-31,991.00,380.11,210.00
4,2018-03-31,1004.00,382.83,209.92
5,2018-06-30,1033.50,393.17,209.85
6,2018-12-31,1025.11,391.44,209.77
7,2019-12-31,1079.17,414.17,209.70
8,2020-12-31,1091.50,407.25,209.62


In [436]:
newpred1 = newpred[x_list]
newpred2 = newpred[x_list]
newpred2

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU
0,1021.73,357.90,245.00
1,1017.90,369.67,220.00
2,1008.20,372.44,190.00
3,991.00,380.11,210.00
4,1004.00,382.83,209.92
5,1033.50,393.17,209.85
6,1025.11,391.44,209.77
7,1079.17,414.17,209.70
8,1091.50,407.25,209.62


In [437]:
newpred = np.array(newpred)
newpred = pd.DataFrame(newpred)
newpred.columns = ['Date'] + x_list
newpred['Date'] = pd.to_datetime(newpred['Date'])
newpred_predicted1 = forest1.predict(newpred1)
df_new_pred1 = pd.DataFrame(newpred_predicted1)
newpred['RDF Model'] = df_new_pred1
newpred_predicted2 = lm8.predict(newpred2)
df_new_pred2 = pd.DataFrame(newpred_predicted2)
newpred['MLR Model'] = df_new_pred2
        
newpred.index = newpred['Date'].values
newpred = newpred.drop(['Date'],axis=1)
newpred

,S_FRONT_CLOSE_EXCH,C_FRONT_CLOSE_EXCH,UREA_GRAN_FLAT_FOB_NOLA_CRU,RDF Model,MLR Model
2017-03-31,1021.73,357.9,245,313.713571,359.385042
2017-06-30,1017.9,369.67,220,320.384762,339.796826
2017-09-30,1008.2,372.44,190,313.981754,315.570160
2017-12-31,991,380.11,210,319.683175,332.804319
2018-03-31,1004,382.83,209.92,319.512223,332.709599
2018-06-30,1033.5,393.17,209.85,318.312760,332.882775
2018-12-31,1025.11,391.44,209.77,316.958760,332.837824
2019-12-31,1079.17,414.17,209.7,316.031403,333.472145
2020-12-31,1091.5,407.25,209.62,315.868546,332.702650


In [438]:
frames = [df_old,newpred]
combined = pd.concat(frames)
date = combined.index
combined['Date'] = date
cols = combined.columns.tolist()
cols = cols[-1:] + cols[:-1]
combined = combined[cols]
combined

,Date,CA-AMNT_TOT_FLAT_CIF_DEU_CRU,C_FRONT_CLOSE_EXCH,DAP_TOT_FLAT_FOB_NOLA_CRU,MLR Model,RDF Model,S_FRONT2_CLOSE_EXCH,S_FRONT_CLOSE_EXCH,UAN_TOT_FLAT_FOB_NOLA_BID_GREEN-MARKETS,UAN_TOT_FLAT_FOB_NOLA_CRU,UREA_GRAN_FLAT_FOB_NOLA_CRU,W_FRONT_CLOSE_EXCH
2014-01-06,2014-01-06,224.606918,427.75,390.468553,389.663053,390.468830,1276.7500,1296.75,257.920000,213.610063,281.623,605.750000
2014-01-07,2014-01-07,224.606918,426,390.468553,389.491347,390.468830,1276.0000,1299.5,217.050909,213.610063,281.623,602.500000
2014-01-08,2014-01-08,224.606918,417,390.468553,388.829230,390.468830,1269.2500,1300.75,217.050909,213.610063,281.623,588.750000
2014-01-09,2014-01-09,235.000000,412,382.500000,467.138152,440.965706,1273.7500,1296.25,217.050909,254.880000,377.5,584.250000
2014-01-10,2014-01-10,224.606918,432.75,390.468553,389.899038,390.468830,1278.5000,1303.75,256.000000,213.610063,281.623,569.000000
2014-01-13,2014-01-13,224.606918,434.5,390.468553,389.629471,390.468830,1294.2500,1326.75,260.160000,213.610063,281.623,573.500000
2014-01-14,2014-01-14,224.606918,431.5,390.468553,389.205979,390.468830,1307.0000,1339,217.050909,213.610063,281.623,579.250000
2014-01-15,2014-01-15,224.606918,425.75,390.468553,389.156519,390.468830,1297.5000,1318,217.050909,213.610063,281.623,567.750000
2014-01-16,2014-01-16,237.500000,428,432.500000,479.431226,463.336304,1296.2500,1315,217.050909,266.880000,391.5,572.750000
2014-01-17,2014-01-17,224.606918,424,390.468553,389.057644,390.468830,1297.2500,1316.5,256.000000,213.610063,281.623,563.500000
